In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.float_format', '{:.10f}'.format)

train = pd.read_csv('data/train.csv')
historical_transactions = pd.read_csv('data/historical_transactions.csv').fillna('')
new_merchant_transactions = pd.read_csv('data/new_merchant_transactions.csv').fillna('')

historical_transactions['purchase_amount'] = np.round(historical_transactions['purchase_amount'] / 0.00150265118 + 497.06,2)
new_merchant_transactions['purchase_amount'] = np.round(new_merchant_transactions['purchase_amount'] / 0.00150265118 + 497.06,2)

In [2]:
missing = historical_transactions[historical_transactions['merchant_id']==''].shape[0]
total = historical_transactions.shape[0]
print(f"Missing: {missing}, total: {total}, missing ratio: {missing/total}")

Missing: 138481, total: 29112361, missing ratio: 0.004756776683278969


In [3]:
missing = historical_transactions[(historical_transactions['merchant_id']=='')\
                                  & (historical_transactions['month_lag']==0)].shape[0]
total = historical_transactions[(historical_transactions['month_lag']==0)].shape[0]
print(f"Missing: {missing}, total: {total}, missing ratio: {missing/total}")

Missing: 66105, total: 3470946, missing ratio: 0.019045240116095152


In [4]:
historical_transactions[historical_transactions['card_id']=='C_ID_d57e4ddab0'].sort_values('purchase_date').tail(10)

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id
26930712,N,C_ID_d57e4ddab0,-1,Y,1,B,879,M_ID_9139332ccc,-5,26.9000000000,2017-09-14 16:23:01,,-1,29
26930639,Y,C_ID_d57e4ddab0,-1,Y,1,B,879,M_ID_9139332ccc,-5,26.9000000000,2017-09-17 16:22:52,,-1,29
26930654,Y,C_ID_d57e4ddab0,-1,Y,1,B,879,M_ID_9139332ccc,-4,26.9000000000,2017-10-14 16:23:03,,-1,29
26930691,Y,C_ID_d57e4ddab0,-1,Y,1,B,879,M_ID_9139332ccc,-3,26.9000000000,2017-11-14 16:22:57,,-1,29
26930690,N,C_ID_d57e4ddab0,-1,Y,1,B,879,M_ID_9139332ccc,-2,26.9000000000,2017-12-14 14:22:57,,-1,29
26930668,N,C_ID_d57e4ddab0,-1,Y,1,B,879,M_ID_9139332ccc,-2,26.9000000000,2017-12-17 14:23:00,,-1,29
26930673,N,C_ID_d57e4ddab0,-1,Y,1,B,879,M_ID_9139332ccc,-2,26.9000000000,2017-12-20 14:22:50,,-1,29
26930705,Y,C_ID_d57e4ddab0,-1,Y,1,B,879,M_ID_9139332ccc,-2,26.9000000000,2017-12-23 14:22:58,,-1,29
26930644,Y,C_ID_d57e4ddab0,-1,Y,1,B,879,M_ID_9139332ccc,-1,26.9000000000,2018-01-14 14:22:54,,-1,29
26930707,Y,C_ID_d57e4ddab0,-1,Y,1,B,574,,0,26.9000000000,2018-02-14 12:22:53,,-1,22


In [5]:
new_merchant_transactions[new_merchant_transactions['card_id']=='C_ID_d57e4ddab0'].sort_values('purchase_date')

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id
1909664,Y,C_ID_d57e4ddab0,-1,Y,1,B,574,M_ID_0855bda176,1,26.9000000000,2018-03-14 10:22:50,,-1,22


In [6]:
fields = ['card_id','city_id','category_1','installments','category_3',\
          'merchant_category_id','category_2','state_id','subsector_id']
new_merchants = new_merchant_transactions[fields + ['merchant_id']].drop_duplicates()
new_merchants = new_merchants.loc[new_merchants['merchant_id']!='']

In [7]:
# take only unique merchants for the `fields` combination
uq_new_merchants = new_merchants.groupby(fields)['merchant_id'].count().reset_index(name = 'n_merchants')
uq_new_merchants = uq_new_merchants.loc[uq_new_merchants['n_merchants']==1]
uq_new_merchants = uq_new_merchants.merge(new_merchants, on = fields)
uq_new_merchants.drop('n_merchants', axis=1, inplace=True)

# rename the merchant_id so we can join it more easily later on
uq_new_merchants.columns = fields + ['imputed_merchant_id']

uq_new_merchants.head()

,card_id,city_id,category_1,installments,category_3,merchant_category_id,category_2,state_id,subsector_id,imputed_merchant_id
0,C_ID_00007093c1,69,N,1,B,879,1.0000000000,9,29,M_ID_00a6ca8a8a
1,C_ID_00007093c1,76,N,1,B,222,3.0000000000,2,21,M_ID_08f01305af
2,C_ID_0001238066,-1,Y,1,B,839,,-1,29,M_ID_e5374dabc0
3,C_ID_0001238066,-1,Y,6,C,771,,-1,31,M_ID_4635824091
4,C_ID_0001238066,69,N,-1,,87,1.0000000000,9,27,M_ID_b34bebaf3c


In [8]:
historical_transactions = historical_transactions.merge(uq_new_merchants, on = fields, how = 'left')

In [9]:
# make the actual imputation for the merchant_id field
historical_transactions.loc[(historical_transactions['merchant_id']=='') & (~pd.isnull(historical_transactions['imputed_merchant_id'])), 'merchant_id'] = \
historical_transactions.loc[(historical_transactions['merchant_id']=='') & (~pd.isnull(historical_transactions['imputed_merchant_id'])), 'imputed_merchant_id']

In [10]:
historical_transactions[historical_transactions['card_id']=='C_ID_d57e4ddab0'].sort_values('purchase_date').tail(10)

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id,imputed_merchant_id
26930712,N,C_ID_d57e4ddab0,-1,Y,1,B,879,M_ID_9139332ccc,-5,26.9000000000,2017-09-14 16:23:01,,-1,29,NaN
26930639,Y,C_ID_d57e4ddab0,-1,Y,1,B,879,M_ID_9139332ccc,-5,26.9000000000,2017-09-17 16:22:52,,-1,29,NaN
26930654,Y,C_ID_d57e4ddab0,-1,Y,1,B,879,M_ID_9139332ccc,-4,26.9000000000,2017-10-14 16:23:03,,-1,29,NaN
26930691,Y,C_ID_d57e4ddab0,-1,Y,1,B,879,M_ID_9139332ccc,-3,26.9000000000,2017-11-14 16:22:57,,-1,29,NaN
26930690,N,C_ID_d57e4ddab0,-1,Y,1,B,879,M_ID_9139332ccc,-2,26.9000000000,2017-12-14 14:22:57,,-1,29,NaN
26930668,N,C_ID_d57e4ddab0,-1,Y,1,B,879,M_ID_9139332ccc,-2,26.9000000000,2017-12-17 14:23:00,,-1,29,NaN
26930673,N,C_ID_d57e4ddab0,-1,Y,1,B,879,M_ID_9139332ccc,-2,26.9000000000,2017-12-20 14:22:50,,-1,29,NaN
26930705,Y,C_ID_d57e4ddab0,-1,Y,1,B,879,M_ID_9139332ccc,-2,26.9000000000,2017-12-23 14:22:58,,-1,29,NaN
26930644,Y,C_ID_d57e4ddab0,-1,Y,1,B,879,M_ID_9139332ccc,-1,26.9000000000,2018-01-14 14:22:54,,-1,29,NaN
26930707,Y,C_ID_d57e4ddab0,-1,Y,1,B,574,M_ID_0855bda176,0,26.9000000000,2018-02-14 12:22:53,,-1,22,M_ID_0855bda176


In [11]:
missing = historical_transactions[(historical_transactions['merchant_id']=='')\
                                  & (historical_transactions['month_lag']==0)].shape[0]
total = historical_transactions[(historical_transactions['month_lag']==0)].shape[0]
print(f"Missing: {missing}, total: {total}, missing ratio: {missing/total}")

Missing: 56200, total: 3470946, missing ratio: 0.016191551237040278


In [15]:
historical_transactions.to_csv('historical_transactions_up.csv', index=False)